In [1]:
from numba import cuda
cuda.select_device(0)
cuda.close()
print('CUDA memory released: GPU0')

CUDA memory released: GPU0


In [2]:
#import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope

In [3]:
from models.unets import Unet2D
from models.deeplab import Deeplabv3, relu6, DepthwiseConv2D, BilinearUpsampling
#from models.separable_unet import Separable_Unet2D
from models.FCN import FCN_Vgg16_16s
from models.SegNet import SegNet

In [4]:
from utils.learning.metrics import dice_coef, precision, recall
from utils.learning.losses import dice_coef_loss
from utils.io.data import DataGen, save_results, save_history, load_data

In [5]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


2023-02-21 17:11:29.343245: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.350282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.350464: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Check the system information

In [6]:
#######################################################################################################
#check the system information, check if cuda and gpu computing for tensorflow is installed properly
import tensorflow as tf
#tf.compat.v1.disable_v2_behavior()
import tensorflow.keras as keras
from tensorflow.python.client import device_lib

print("whether tensorflow is built with cuda: ", tf.test.is_built_with_cuda())
print("whether gpu computing is available for tensorflow: ", tf.test.is_gpu_available())
#print("whether gpu computing is available for tensorflow: ",tf.config.list_physical_devices('GPU'))
print("using keras version: ", keras.__version__)
print("using tensorflow version: ", tf.__version__)
print("\n")
print("Device details:\n", device_lib.list_local_devices())
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

whether tensorflow is built with cuda:  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


2023-02-21 17:11:29.360122: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


whether gpu computing is available for tensorflow:  True
using keras version:  2.9.0
using tensorflow version:  2.9.1


Device details:
 [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3649222401141645586
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23628349440
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16488670866268821752
physical_device_desc: "device: 0, name: NVIDIA TITAN RTX, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]
Default GPU Device: /device:GPU:0


2023-02-21 17:11:29.360760: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.361061: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.361193: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.830065: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.830308: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [7]:
# For TF 1.0, set the GPU memory growth
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
#config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.8
#config.gpu_options.allow_growth = True
#sess = tf.InteractiveSession(config=config)

# For TF 2.0, set the GPU memory growth
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
#from tensorflow import ConfigProto
#from tensorflow import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Instructions for updating:
non-resource variables are not supported in the long term


2023-02-21 17:11:29.846135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.846519: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.846776: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.847092: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-21 17:11:29.847371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [8]:
# Varibales and data generator
input_dim_x=224
input_dim_y=224
n_filters = 32
dataset = 'benchmark_datasets/azh_wound_dataset'#'merged_azh_dog1_8_2800'#'dog1_8_1400'#'dog1_8'#'azh_wound_care_center_dataset_patches'#'Medetec_foot_ulcer_224'#'merged_azh_dog1_8'
data_gen = DataGen('./data/' + dataset + '/', split_ratio=0.2, x=input_dim_x, y=input_dim_y)

In [9]:
######### Get the deep learning models #########

######### Unet ##########
#unet2d = Unet2D(n_filters=n_filters, input_dim_x=None, input_dim_y=None, num_channels=3)
# unet-1
#model, model_name = unet2d.get_unet_model_yuanqing()
# unet-2
#model, model_name = unet2d.get_unet_2_model()

######### Seperable Unet ##########
# Not working now, has package import issue
# sep_unet = Seperable_Unet2D(n_filters=n_filters, input_dim_x=input_dim_x, input_dim_y=input_dim_y, num_channels=3)
# model, model_name = sep_unet.get_seperable_unet_model_5_levels()
# model, model_name = sep_unet.get_sep_unet_v3()

######### MobilenetV2 and DeepLabV3+#####################
# Not working now, has package import issue
#model = Deeplabv3(input_shape=(input_dim_x, input_dim_y, 3), classes=1)
#model_name = 'DeepLabV3+'
#with CustomObjectScope({'relu6': relu6,'DepthwiseConv2D': DepthwiseConv2D, 'BilinearUpsampling': BilinearUpsampling}):
#    model = load_model('azh_wound_care_center_diabetic_foot_training_history/2020-02-10 02:57:27.555495.hdf5'
#                       , custom_objects={'dice_coef': dice_coef, 'precision':precision, 'recall':recall})

######### Vgg16 ##########
#model, model_name = FCN_Vgg16_16s(input_shape=(input_dim_x, input_dim_y, 3))

######### SegNet ##########
segnet = SegNet(n_filters, input_dim_x, input_dim_y, num_channels=3)
# SegNet-1
#model, model_name = segnet.get_SegNet()
# SegNet-2
#model, model_name = segnet.get_SegNet_2()

# SegNet-3
model, model_name = segnet.get_SegNet_3()

#plot_model(model, to_file=model_name+'.png')

In [10]:
# training
batch_size = 2
epochs = 2000
learning_rate = 1e-4
loss = 'binary_crossentropy'

# restore_best_weights need at least tf version 1.15 and keras version 2.2.3
# currently using tf 1.10 and keras 2.1.3
#es = EarlyStopping(monitor='val_dice_coef', patience=200, mode='max', restore_best_weights=True)
es = EarlyStopping(monitor='val_dice_coef', patience=200, mode='max')
#training_history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs
#                             , validation_split=0.2, verbose=1, callbacks=[])

model.summary()
model.compile(optimizer=Adam(lr=learning_rate), loss=loss, metrics=[dice_coef, precision, recall])



Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 conv2d (Conv2D)                (None, 224, 224, 64  15616       ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling_with_argmax2d (Max  [(None, 112, 112, 6  0          ['conv2d[0][0]']                 
 PoolingWithArgmax2D)           4),                                                               
                                 (None, 112, 112, 6                                               
                                4)]                                                               
                                                                                              

/home/aravind/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [11]:
training_history = model.fit(data_gen.generate_data(batch_size=batch_size, train=True),
                                       steps_per_epoch=int(data_gen.get_num_data_points(train=True) / batch_size),
                                       callbacks=[es],
                                       validation_data=data_gen.generate_data(batch_size=batch_size, val=True),
                                       validation_steps=int(data_gen.get_num_data_points(val=True) / batch_size),
                                       epochs=epochs,
                                       max_queue_size = 1,
                                       use_multiprocessing=False)

2023-02-21 17:11:31.704455: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled


Epoch 1/2000


2023-02-21 17:11:32.854183: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


443/444 [============================>.] - ETA: 0s - batch: 221.0000 - size: 2.0000 - loss: 0.6072 - dice_coef: 0.0061 - precision: 0.1737 - recall: 0.0048

/home/aravind/anaconda3/lib/python3.9/site-packages/keras/engine/training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


...

444/444 [==============================] - 7s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0790 - dice_coef: 0.5184 - precision: 0.9317 - recall: 0.3972 - val_loss: 0.3253 - val_dice_coef: 0.3754 - val_precision: 0.6228 - val_recall: 0.3151
Epoch 312/2000
444/444 [==============================] - 6s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0789 - dice_coef: 0.5191 - precision: 0.9336 - recall: 0.3980 - val_loss: 0.3216 - val_dice_coef: 0.3770 - val_precision: 0.6080 - val_recall: 0.3203
Epoch 313/2000
444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0790 - dice_coef: 0.5238 - precision: 0.9340 - recall: 0.3982 - val_loss: 0.3330 - val_dice_coef: 0.3839 - val_precision: 0.6387 - val_recall: 0.3222
Epoch 314/2000
444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0789 - dice_coef: 0.5221 - precision: 0.9341 - recall: 0.3987 - val_loss: 0.3330 - val_dice_coef: 0.3816 - va

444/444 [==============================] - 7s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0798 - dice_coef: 0.5176 - precision: 0.9352 - recall: 0.3905 - val_loss: 0.3225 - val_dice_coef: 0.3725 - val_precision: 0.6538 - val_recall: 0.3038
Epoch 343/2000
444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0796 - dice_coef: 0.5174 - precision: 0.9358 - recall: 0.3928 - val_loss: 0.3280 - val_dice_coef: 0.3656 - val_precision: 0.6541 - val_recall: 0.2963
Epoch 344/2000
444/444 [==============================] - 7s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0796 - dice_coef: 0.5195 - precision: 0.9351 - recall: 0.3925 - val_loss: 0.3330 - val_dice_coef: 0.3615 - val_precision: 0.6618 - val_recall: 0.2906
Epoch 345/2000
444/444 [==============================] - 6s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0796 - dice_coef: 0.5193 - precision: 0.9373 - recall: 0.3923 - val_loss: 0.3271 - val_dice_coef: 0.3613 - va

444/444 [==============================] - 6s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0794 - dice_coef: 0.5192 - precision: 0.9345 - recall: 0.3948 - val_loss: 0.3459 - val_dice_coef: 0.3818 - val_precision: 0.6125 - val_recall: 0.3242
Epoch 374/2000
444/444 [==============================] - 7s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0796 - dice_coef: 0.5178 - precision: 0.9344 - recall: 0.3932 - val_loss: 0.3647 - val_dice_coef: 0.3796 - val_precision: 0.6129 - val_recall: 0.3222
Epoch 375/2000
444/444 [==============================] - 6s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0798 - dice_coef: 0.5142 - precision: 0.9317 - recall: 0.3921 - val_loss: 0.3407 - val_dice_coef: 0.3788 - val_precision: 0.6425 - val_recall: 0.3129
Epoch 376/2000
444/444 [==============================] - 7s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0797 - dice_coef: 0.5175 - precision: 0.9365 - recall: 0.3928 - val_loss: 0.3553 - val_dice_coef: 0.3689 - va

444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0802 - dice_coef: 0.5160 - precision: 0.9370 - recall: 0.3888 - val_loss: 0.3030 - val_dice_coef: 0.3704 - val_precision: 0.6437 - val_recall: 0.3029
Epoch 405/2000
444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0799 - dice_coef: 0.5162 - precision: 0.9347 - recall: 0.3913 - val_loss: 0.3248 - val_dice_coef: 0.3508 - val_precision: 0.6959 - val_recall: 0.2722
Epoch 406/2000
444/444 [==============================] - 7s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0800 - dice_coef: 0.5150 - precision: 0.9379 - recall: 0.3900 - val_loss: 0.3399 - val_dice_coef: 0.3447 - val_precision: 0.7050 - val_recall: 0.2651
Epoch 407/2000
444/444 [==============================] - 6s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0795 - dice_coef: 0.5197 - precision: 0.9386 - recall: 0.3949 - val_loss: 0.3584 - val_dice_coef: 0.3705 - va

444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0797 - dice_coef: 0.5208 - precision: 0.9379 - recall: 0.3934 - val_loss: 0.3597 - val_dice_coef: 0.3618 - val_precision: 0.6789 - val_recall: 0.2875
Epoch 436/2000
444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0799 - dice_coef: 0.5170 - precision: 0.9399 - recall: 0.3914 - val_loss: 0.3721 - val_dice_coef: 0.3651 - val_precision: 0.6662 - val_recall: 0.2921
Epoch 437/2000
444/444 [==============================] - 6s 14ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0797 - dice_coef: 0.5209 - precision: 0.9405 - recall: 0.3931 - val_loss: 0.3748 - val_dice_coef: 0.3662 - val_precision: 0.6690 - val_recall: 0.2971
Epoch 438/2000
444/444 [==============================] - 7s 15ms/step - batch: 221.5000 - size: 2.0000 - loss: 0.0795 - dice_coef: 0.5225 - precision: 0.9407 - recall: 0.3949 - val_loss: 0.3972 - val_dice_coef: 0.3683 - va

In [14]:
tf.saved_model.save(model,'my_model')

2023-02-22 12:53:24.011430: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 12:53:24.011684: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 12:53:24.011839: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 12:53:24.012029: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-22 12:53:24.012175: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [15]:
loaded_model = tf.saved_model.load('my_model')

In [28]:
@tf.function
def compare_models(model1, model2):
    for w1, w2 in zip(model1.variables, model2.variables):
        if not tf.math.equal(w1,w2):
            print('The models are different')
            break
    else:
        print('models are Same')

In [29]:
compare_models(model,loaded_model)

Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


OperatorNotAllowedInGraphError: Using a symbolic `tf.Tensor` as a Python `bool` is not allowed: AutoGraph did convert this function. This might indicate you are trying to use an unsupported feature.

In [16]:
### save the model weight file and its training history
save_history(model, model_name, training_history, dataset, n_filters, epochs, learning_rate, loss, color_space='RGB',
             path='./trained_models/')

dog_data


KeyError: 'input_1_ib-0'

In [23]:
from numba import cuda
cuda.select_device(0)
cuda.close()
print('CUDA memory released: GPU0')

CUDA memory released: GPU0
